In [165]:
import pandas as pd
import pyodbc
import os
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

from datetime import datetime
import numpy as np

In [166]:
print("Script Run Started")

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:/Users/Administrator/LIS/Dabur/python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

Script Run Started


In [167]:
# Open Sheet
sheet = client.open_by_key("1U1yF1gpdUNLwsDMbJGUJhhSAVXT4opjpc2tKy5aj750") # Master Sheets ( Dimensions Sheet)
Data = sheet.worksheet("Data")
UserSheet = sheet.worksheet("User")
Attendance = sheet.worksheet("Attendance")

In [168]:
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
    """

In [169]:
x=['ReckittManagement','ZydusMTManagement','AmulManagement','CholayilManagement','BrillonMTManagement','DanoneMTManagement','EvereadyManagement'
   ,'HaleonManagement','MTRManagement','IDAMManagement','GCPL_SAMTManagement','PidiliteManagement','RelianceManagement',
   'AdaniWilmerManagement','PiramalManagement','Jayanti_GroupManagement',
   'EmamiGTManagement','BeamSuntoryManagement','WiproGranammaManagement','DanoneGTManagement',
   'HimalayaMTManagement','DaburPharmaManagement','JyothyLabsManagement','GCPLManagement','MarsManagement',
   'HimalayaTurmericManagement','HimalayaTurmericGTManagement','ParleManagement',
   'JohnsonJohnsonManagement','KRBLMTManagement','ZydusGTManagement','BajajConsumerManagement',
   'BajajMTManagement','DelMonteManagement','GCPLFoodsManagement','ColgateManagement','BacardilegacyManagement',
   'AbbottManagement','BacardiPremiumManagement','EmamiAgroManagement','KRBLGTManagement','GodrejNinjaFoodsManagement',
   'LBPLManagement']
# x=['ReckittManagement']

In [170]:
conn = pyodbc.connect(conn_str)
visitdf = pd.DataFrame()

In [171]:
for value in x:
    query = f"""
    WITH CTE AS
    (
        SELECT
            (TXT.UserName + '-' + TXT.Visit_Date) AS UserVisitKey,
            TXT.Visit_Date,
            TXT.UserName,
            (TXT.Visit_Date + ' ' + TXT.CHEKINTIME) AS ChekinTime,
            TXT.LISStoreCode,
            (TXT.Visit_Date + '-' + TXT.LISStoreCode + '-' + TXT.UserName) AS StoreDateKey,
            TXT.StoreName,
            TXT.Designation,
            (TXT.Visit_Date + ' ' + TXT.CHECKOUTTIME) AS ChekoutTime,
            TXT.FullName,
            TXT.NoPermissionReason,
            TXT.Region,
            TXT.State,
            TXT.City,
            TXT.Status,
            TXT.Client as EmployeeID
        FROM
        (
            SELECT
                '{value}' AS Client,
                FORMAT(CONVERT(DATETIME, SA.[VisitDate], 105), 'dd-MM-yyyy') AS Visit_Date,
                SA.LISStoreCode,
                SA.StoreName,
                SA.UserName,
                SA.Designation,
                SA.FullName,
                CONVERT(VARCHAR, CAST([CheckIn-Time] AS TIME), 8)  AS CHEKINTIME,
                CONVERT(VARCHAR, CAST([CheckOut-Time] AS TIME), 8) AS CHECKOUTTIME,
                SA.NoPermissionReason,
                UM.Region,
                UM.State,
                UM.City,
                UM.Status,
                UM.EmployeeID
            FROM [PowerBI_LISV2].[{value}].[ISPStoreAttendance_NormalExport] AS SA
            LEFT JOIN [PowerBI_LISV2].[{value}].UserMaster AS UM
                ON UM.UserName = SA.UserName
            WHERE
                -- [Month] = MONTH(GETDATE() - 1)
                -- AND [Year]  = YEAR(GETDATE() - 1)
                DATEFROMPARTS([Year], [Month], 1) >= DATEADD(MONTH, -5, DATEFROMPARTS(YEAR(GETDATE()), MONTH(GETDATE()), 1))
                AND UM.Status = 'True'
                AND SA.UserName NOT LIKE '%Star%'
                AND SA.UserName NOT LIKE '%TEST%'
                AND SA.UserName LIKE '%SUP%'
        ) AS TXT
    )
    SELECT *
    FROM CTE;
    """

    print(f"Running for: {value}")
    df = pd.read_sql(query, conn)
    visitdf = pd.concat([visitdf, df], ignore_index=True)


Running for: ReckittManagement


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41520\2523976501.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Running for: ZydusMTManagement
Running for: AmulManagement
Running for: CholayilManagement
Running for: BrillonMTManagement
Running for: DanoneMTManagement
Running for: EvereadyManagement
Running for: HaleonManagement
Running for: MTRManagement
Running for: IDAMManagement
Running for: GCPL_SAMTManagement
Running for: PidiliteManagement
Running for: RelianceManagement
Running for: AdaniWilmerManagement
Running for: PiramalManagement
Running for: Jayanti_GroupManagement
Running for: EmamiGTManagement
Running for: BeamSuntoryManagement


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41520\2523976501.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41520\2523976501.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Running for: WiproGranammaManagement
Running for: DanoneGTManagement
Running for: HimalayaMTManagement
Running for: DaburPharmaManagement


...

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41520\2523976501.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Running for: MarsManagement
Running for: HimalayaTurmericManagement
Running for: HimalayaTurmericGTManagement
Running for: ParleManagement
Running for: JohnsonJohnsonManagement
Running for: KRBLMTManagement
Running for: ZydusGTManagement
Running for: BajajConsumerManagement
Running for: BajajMTManagement
Running for: DelMonteManagement
Running for: GCPLFoodsManagement


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41520\2523976501.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41520\2523976501.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Running for: ColgateManagement
Running for: BacardilegacyManagement
Running for: AbbottManagement
Running for: BacardiPremiumManagement
Running for: EmamiAgroManagement
Running for: KRBLGTManagement
Running for: GodrejNinjaFoodsManagement
Running for: LBPLManagement


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41520\2523976501.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [172]:
# UserDataQuery ="""
# WITH CTE AS
# (    SELECT 
#         UM2.UserMasterId,
# 		UM2.EmployeeID,
#         UM2.UserName,
#         UM2.Status,
# 		UM2.FullName,
# 		UM2.Designation,
#         UM2.LastWorkingDate,
#         UM2.Region,
# 		UM2.State,
#         UM2.City,
#         RANK() OVER (PARTITION BY UM2.UserName ORDER BY UM2.UserMasterId DESC) AS RK
#     FROM [PowerBI_LISV2].[GCPLFoodsManagement].UserMaster AS UM2
#     -- WHERE UM2.UserName IN ('GCPLF-ISP-136')
# 	)
# SELECT CTE.EmployeeID,CTE.UserName,CTE.Status,CTE.FullName,CTE.Designation,CTE.Region,CTE.State,CTE.City,
# SUP.Supervisor
# FROM CTE
# LEFT JOIN
# 	(SELECT ISP,Supervisor FROM
# 		(SELECT *,RANK() OVER (PARTITION BY UH.ISP ORDER BY UH.UserHierarchyListId DESC) AS RK FROM
# 			[PowerBI_LISV2].[GCPLFoodsManagement].UserHierarchyListReport AS UH)AS UH WHERE UH.RK =1) AS SUP
# 			ON SUP.ISP = CTE.UserName
# WHERE CTE.RK = 1 AND CTE.Status = 'True' and 
# (CTE.UserName like '%ISP%' OR CTE.UserName like '%MR%' OR CTE.UserName like '%SUP%');

# """ 



In [173]:
# query2attendance = """
# WITH Attendance_CTE AS (
#     SELECT 
#         A.UserName,
#         A.FullName,
#         A.Region,
#         A.Designation,
#         ''as PositionCode,
#         CASE
#             WHEN A.Status = 'Present' THEN 'P'
#             WHEN A.Status = 'Absent' THEN 'A'
#             WHEN A.Status = 'Leave' THEN 'L'
#             WHEN A.Status = 'Compensatory Off' THEN 'CO'
#             WHEN A.Status IN ('Week Off', 'WeekOff') THEN 'WO'
#             WHEN A.Status = 'Meeting' THEN 'M'
#             WHEN A.Status = 'Training' THEN 'T'
#             WHEN A.Status IN ('National Hoilday') THEN 'H'
#             WHEN A.Status = 'Holiday' THEN 'H'
#             ELSE 'NA'
#         END AS AttStatus,
#         DAY(TRY_CONVERT(date, A.[Date], 105)) AS DayNum
#     FROM PowerBI_LISV2.[GCPLFoodsManagement].ISPAttendanceMasterExport AS A
#     LEFT JOIN PowerBI_LISV2.[GCPLFoodsManagement].UserMaster AS M
#         ON A.UserName = M.UserName
#     WHERE TRY_CONVERT(date, A.[Date], 105) >= 
#     CASE 
#         WHEN DAY(GETDATE()) = 1 
#         THEN DATEFROMPARTS(YEAR(DATEADD(MONTH, -1, GETDATE())), MONTH(DATEADD(MONTH, -1, GETDATE())), 1)
#         ELSE DATEFROMPARTS(YEAR(GETDATE()), MONTH(GETDATE()), 1)
#     END
#         AND TRY_CONVERT(date, A.[Date], 105) < 
#     CASE 
#         WHEN DAY(GETDATE()) = 1 
#         THEN DATEADD(MONTH, 1, DATEFROMPARTS(YEAR(DATEADD(MONTH, -1, GETDATE())), MONTH(DATEADD(MONTH, -1, GETDATE())), 1))
#         ELSE DATEADD(MONTH, 1, DATEFROMPARTS(YEAR(GETDATE()), MONTH(GETDATE()), 1))
#     END


#         AND M.Status = 'True'
#         AND (A.UserName LIKE '%ISP%' OR A.UserName LIKE '%MR%')
# ),
# Pivoted AS (
#     SELECT *
#     FROM Attendance_CTE
#     PIVOT (
#         MAX(AttStatus)
#         FOR DayNum IN (
#             [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
#             [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
#             [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31]
#         )
#     ) AS P
# )
# SELECT *
# FROM Pivoted
# ORDER BY UserName;

    
# """

In [174]:
# conn = pyodbc.connect(conn_str)
# df = pd.read_sql(query, conn)
# df_attendance = pd.read_sql(query2attendance, conn)


In [175]:
# UserData = pd.read_sql(UserDataQuery, conn)
# Supervisor = UserData[UserData['Designation']=='Supervisor']
# Supervisor2 = Supervisor[['UserName','FullName']]
# Supervisor2 = Supervisor2.rename(columns={'UserName':'Supervisor','FullName':'SupervisorName'})
# Supervisor2
# dfUser = pd.merge(UserData,Supervisor2,left_on = 'Supervisor', right_on= 'Supervisor', how='left')

# UserData2=dfUser.drop(columns = ['FullName','Designation'])

In [176]:

print("data extracted....")
print("data extracted....")

loginlogout = visitdf #alternate
# loginlogout = pd.merge(df,UserData2, how = 'left',on = 'UserName')


data extracted....
data extracted....


In [177]:
loginlogout.head()

UserVisitKey  Visit_Date   UserName           ChekinTime  \
0  RB-SUP-13-01-08-2025  01-08-2025  RB-SUP-13  01-08-2025 17:37:47   
1  RB-SUP-30-01-08-2025  01-08-2025  RB-SUP-30  01-08-2025 17:05:13   
2  RB-SUP-04-01-08-2025  01-08-2025  RB-SUP-04  01-08-2025 17:04:24   
3  RB-SUP-24-01-08-2025  01-08-2025  RB-SUP-24  01-08-2025 16:49:34   
4  RB-SUP-05-01-08-2025  01-08-2025  RB-SUP-05  01-08-2025 16:46:25   

  LISStoreCode                    StoreDateKey  \
0    RB00-3510  01-08-2025-RB00-3510-RB-SUP-13   
1      RB00-20    01-08-2025-RB00-20-RB-SUP-30   
2    RB00-3663  01-08-2025-RB00-3663-RB-SUP-04   
3    RB00-5827  01-08-2025-RB00-5827-RB-SUP-24   
4    RB00-2886  01-08-2025-RB00-2886-RB-SUP-05   

                               StoreName Designation          ChekoutTime  \
0                   4016 Seawoods D-Mart  Supervisor  01-08-2025 18:23:31   
1                          Ananda Export  Supervisor  01-08-2025 18:12:18   
2                     4101 Karjat D-Mart  Supervisor  01-08-2025 19:16:56   
3  Smart Bazaar- Paras Downtown Zirakpur  Supervisor  01-08-2025 20:24:20   
4                    Aero Business Tower  Supervisor  01-08-2025 17:18:23   

                 FullName NoPermissionReason Region        State       City  \
0  Munil Lalbahadur Yadav               None   West  Maharashtra     Mumbai   
1              Tarun Gope               None   East    Jharkhand     Ranchi   
2     Ganesh Janardan Lad               None   West  Maharashtra     Mumbai   
3           Puneet Tiwari               None  North       Punjab  Jalandhar   
4       Gaurav Kumar Vyas               None  North        Delhi      Delhi   

  Status         EmployeeID  
0   True  ReckittManagement  
1   True  ReckittManagement  
2   True  ReckittManagement  
3   True  ReckittManagement  
4   True  ReckittManagement

In [178]:
loginlogout['ChekinRank'] = loginlogout.groupby(['StoreDateKey'])['ChekinTime'].rank(method='dense', ascending=True).astype(int)
loginlogout2= loginlogout[loginlogout['ChekinRank']==1]

In [179]:
loginlogout2=loginlogout2.drop(columns = ['ChekinRank'])

In [180]:
loginlogout2 = loginlogout2.drop_duplicates()
loginlogout2.at[0, 'LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [181]:
loginlogout2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 122411 entries, 0 to 138341
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   UserVisitKey        122411 non-null  object
 1   Visit_Date          122411 non-null  object
 2   UserName            122411 non-null  object
 3   ChekinTime          122411 non-null  object
 4   LISStoreCode        122411 non-null  object
 5   StoreDateKey        122411 non-null  object
 6   StoreName           122411 non-null  object
 7   Designation         122411 non-null  object
 8   ChekoutTime         122410 non-null  object
 9   FullName            122411 non-null  object
 10  NoPermissionReason  7512 non-null    object
 11  Region              122411 non-null  object
 12  State               122411 non-null  object
 13  City                122411 non-null  object
 14  Status              122411 non-null  object
 15  EmployeeID          122411 non-null  object
 16  LastUpd

In [182]:
# print("csv updated....")
# loginlogout2.to_csv(r'C:\Users\Administrator\LIS\Dabur\gcfoodsloginlogout.csv', index=False)

loginlogout2 = loginlogout2.drop_duplicates()
loginlogout2= loginlogout2[['UserVisitKey','Visit_Date','UserName','ChekinTime','LISStoreCode','StoreDateKey','StoreName','Designation',
                           'ChekoutTime','FullName','NoPermissionReason','EmployeeID',
                           'Region','State','City','Status','LastUpdated']]

# 'Supervisor','SupervisorName'

In [183]:
#Sending df on sheet
# Data.batch_clear(["A1"])
# set_with_dataframe(Data, loginlogout2)

# UserSheet.batch_clear(["A1"])
# set_with_dataframe(UserSheet, dfUser)

# Attendance.batch_clear(["A1"])
# set_with_dataframe(Attendance, df_attendance)

print("Google Sheets are updated")

Google Sheets are updated


In [184]:
loginlogout2.to_csv('loginlogoutsupervisor_aug_jan.csv')